In [22]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [23]:
hex_scheduled = _hex_json.loads("false")

In [24]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [25]:
hex_user_attributes = _hex_json.loads("{}")

In [26]:
hex_run_context = _hex_json.loads("\"logic\"")

In [27]:
hex_timezone = _hex_json.loads("\"UTC\"")

In [28]:
hex_project_id = _hex_json.loads("\"965f7012-9e09-433f-a10d-b4bc04a1b798\"")

In [29]:
hex_project_name = _hex_json.loads("\"Welcome to Hex!\"")

In [30]:
hex_status = _hex_json.loads("\"\"")

In [31]:
hex_categories = _hex_json.loads("[]")

In [32]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

In [33]:
# import jinja2
# raw_query = """
#     -- Use SQL to get all food orders from a demo data warehouse
#     
# """
# sql_query = jinja2.Template(raw_query).render(vars())

In [34]:
from numpy import load

data = load('/Users/wuruoyu/Sleep-Staging/data/train_data2_noCoordinates.npz')
import pandas as pd
lst = data.files
for item in lst:
    print(item)

# only_sweets = orders[orders["CATEGORY"] == "Sweets"]
# popular_dessert = only_sweets["MENU_ITEM"].mode()[0]
# print(f"The most popular dessert is the {popular_dessert}")

X
y
region


In [35]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
X = list(data["X"]) + list(data["X"])
X = [str(i) for i in X]
y = list(data["y"]) + list(data["y"])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)
X_train_tokenized.keys()
len(X_train),len(X_val)

(5491, 1373)

In [37]:

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx].astype(int))
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, [(i - 1)/2 for i in y_train])
val_dataset = Dataset(X_val_tokenized, [(i - 1)/2 for i in y_val])

train_dataset[5]

{'input_ids': tensor([  101,  1031,  1017,  1012, 14078,  2620, 23352,  2692,  2509,  2063,
          1011,  5757,  1016,  1012,  5139,  2620,  2581, 18827,  2620,  2475,
          2063,  1011,  5757,  1011,  1020,  1012,  4008,  2683,  2575,  2581,
         23632,  2509,  2063,  1011,  5757,  1012,  1012,  1012,  1020,  1012,
          5986,  2620, 22932, 24434,  2487,  2063,  1011,  5718,  1011,  1016,
          1012,  3878,  2581,  2581,  2581,  2475,  2063,  1011,  5757,  1017,
          1012,  6486, 10790, 22907, 20958,  2063,  1011,  5757,  1033,   102,
             0,     0,     0,     0,     0,     0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [38]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [39]:
# ! pip install accelerate>=0.26.0
# ! pip install -U transformers
# Define Trainer

args = TrainingArguments(
    output_dir="output",
    num_train_epochs=10,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [40]:
trainer.train()

  3%|▎         | 230/6870 [04:33<2:37:09,  1.42s/it]

In [20]:
trainer.evaluate()

100%|██████████| 172/172 [00:40<00:00,  4.07it/s]/Users/wuruoyu/anaconda3/envs/transformers/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
100%|██████████| 172/172 [00:41<00:00,  4.19it/s]

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.4792148768901825,
 'eval_accuracy': 0.8201019664967225,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_runtime': 41.3178,
 'eval_samples_per_second': 33.23,
 'eval_steps_per_second': 4.163,
 'epoch': 1.0}